# Autobrains HW
# Intro:
The notebook consists of two parts: infrastructure and usage example.

# Disclaimer:

the infrastructure part was short and easy, but the example part was challenging (and fun) because there wasn't any "off-the-shelf" vision model that easily exposes his intermediate layers outputs, so i had to reverse engineer the `torch.models.detection.fasterrcnn_resnet50_fpn`
forward pass code and add new logic in the original pytorch implementation.

# infrastructure
design for a creation of a subset dataset consistent of "classification issue" (as stated in the pdf)
between multiple labels, along with model that can encapsulate logic (via `lightweight_function()`) that can "solve" the "classification issue".

In [1]:
import torch
import torchvision.models as models
from torchvision.ops import box_iou
import os
import numpy as np
import torch
from PIL import Image, ImageDraw
from torchvision.ops import boxes as box_ops, roi_align
import pickle
from typing import Optional, List, Callable

from model_utils import CustomRCNN, label_map_model
from data_utils import label_map_dataset

/Users/ramshirazi/miniforge3/envs/abEnv0/lib/python3.9/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/Users/ramshirazi/miniforge3/envs/abEnv0/lib/python3.9/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Plea

### BaseDataset Class :
- can receive a list of images targets and predications (target and predication are dicts)
- can iteratively append elements, using `append()` method.

In [2]:
class BaseDataset:
    def __init__(
        self,
        images: Optional[list[torch.Tensor]] = None,
        targets: Optional[list[dict]] = None,
        predictions: Optional[list[dict]] = None,
    ):
        self.images = images if images else []
        self.targets = targets if targets else []
        self.predictions = predictions if predictions else []

    def append(
        self,
        image: torch.Tensor,
        target: dict,
        prediction: dict,
    ):
        self.images.append(image)
        self.targets.append(target)
        if prediction:
            self.predictions.append(prediction)

    @classmethod
    def load_from_pkl(cls, root="./small_dataset.pkl"):
        dataset = cls()
        with open(root, "rb") as f:
            data = pickle.load(f)
        for image, target in data:
            dataset.append(image, target, None)
        return dataset

### Base model class:
  responsible for enveloping a core model that returns the model predications along with a embedding vectors for each predication (the embeddings are the model intermediate layers outputs for each predication)
  - the model can have a multiple bounding box predications for each images.
  - the model outputs the embeddings for each bounding box predication 
  - can also "solve" a predication with a `lightweight_function()` callable, that operate on the model output and embeddings 
  - this model cant be trained 
  -  the model input is a list of RGB images (as a batch)
 #### NOTES: 
 - we cant always stack the batched images to tensor (like shape : `(batch_dim,C,H,W)`) because the images size can vary, so we assume a list of images (tensors) as model input.
  



In [3]:
class BaseNN:
    def __init__(
        self, model: torch.nn.Module, lightweight_function: Optional[Callable] = None
    ):
        self.model = model
        self.lightweight_function = lightweight_function if lightweight_function else None

    def inference(
        self, input_data: list[torch.Tensor]
    ) -> tuple[list[dict], list[torch.Tensor]]:
        with torch.no_grad():
            predictions, embedding_output = self.model(input_data)

        if self.lightweight_function:
            predictions = self.lightweight_function(predictions, embedding_output)

        return predictions, embedding_output

### ClassificationIssueDataset:
inherent from `BaseDataset` with the additions of `self.embeddings` class member 

In [4]:
class ClassificationIssueDataset(BaseDataset):
    def __init__(
        self, embeddings: list[list[torch.Tensor]] = None, *args, **kwargs
    ):
        super().__init__(*args, **kwargs)
        self.embeddings = embeddings if embeddings else []


    def append(self, embeddings: list[torch.Tensor] = None, **kwargs):
        super().append(**kwargs)
        if embeddings is not None:
            self.embeddings.append(embeddings)

### get_incorrect_pairs()
this function encapsulate the logic for detecting a classification issue.

if the pair of bounding boxes have iou > iou_threshold and the labels are not equal
then the pair is incorrect.
 

In [5]:
def get_incorrect_pairs(
    bb_t: torch.Tensor,  # shape: (num_target_boxes, 4)
    bb_p: torch.Tensor,  # shape: (num_predicted_boxes, 4)
    labels_t: torch.Tensor,  # shape: (num_target_boxes)
    labels_p: torch.Tensor,  # shape: (num_predicted_boxes)
    iou_threshold=0.75,
):
    """vectorized implementation using broadcasting
    if the pair of bounding boxes have iou > iou_threshold and the labels are not equal
    then the pair is incorrect"""
    iou_matrix = box_iou(bb_t, bb_p)  # shape: (num_target_boxes, num_predicted_boxes)
    incorrect_labels = torch.logical_not(
        torch.eq(labels_t.unsqueeze(-1), labels_p.unsqueeze(-2))
    )  # using broadcasting
    incorrect_pairs = torch.nonzero(
        torch.logical_and(iou_matrix > iou_threshold, incorrect_labels)
    )
    return incorrect_pairs

### identify_classification_issue()
- first, we are skipping images which dont have the labels in `labels`
- we identify incorrect pairs (with the logic in `get_incorrect_pairs()`)
- if the model predicted label is not in `labels`, we skip this sample.
- we append to a `ClassificationIssueDataset` the relevant images, target, predications and embeddings.

#### NOTES: 
- the function stops when the returned dataset has 3 elements (iterate through the whole dataset takes more then 5 minutes with my computer )
- we could reduce the computation time by filtering the dataset for relevant images only and then run `model.inference(list_of_filtered_images)`, but my computer cant take it (along with creating a pytorch DataLoader)

In [15]:
def identify_classification_issue(
    dataset: BaseDataset, model: BaseNN, labels: list[int], iou_threshold=0.75
) -> ClassificationIssueDataset:
    classification_issue_dataset = ClassificationIssueDataset()

    for image, target in zip(dataset.images, dataset.targets):
        if not any(l in target["labels"] for l in labels):
            continue
        prediction, embeddings = model.inference([image])
        prediction, embeddings = prediction[0], embeddings[0]

        incorrect_pairs = get_incorrect_pairs(
            bb_t=torch.stack(target["boxes"]),
            bb_p=prediction["boxes"],
            labels_t=torch.tensor(target["labels"]),
            labels_p=prediction["labels"],
            iou_threshold=iou_threshold,
        )
                
        if len(incorrect_pairs) > 0:
            prediction_labels = prediction["labels"][incorrect_pairs[:, 1]]
            
            if not any(l in prediction_labels for l in labels):
                continue
            
            classification_issue_dataset.append(
                image=image,
                target={
                    "labels": torch.tensor(target["labels"])[incorrect_pairs[:, 0]],
                    "boxes": torch.stack(target["boxes"])[incorrect_pairs[:, 0]],
                },
                prediction={
                    "labels": prediction_labels,
                    "boxes": prediction["boxes"][incorrect_pairs[:, 1]],
                },
                embeddings=embeddings[incorrect_pairs[:, 1]],
            )
        if len(classification_issue_dataset.images) >= 3:
            break
    return classification_issue_dataset

## lightweight_function()
in order to "solve" the classification issue we can:
- use KNN algorithm with the previously computed embedding vectors, this is the a good classic algorithm that benefits from a fast compute time, good accuracy for big datasets (which we have as stated in the goal2 of the pdf) and no training is required (just find the best K)
- we can use SVM algorithm with the previously computed embedding vectors which can work great for high dimensional embedding vectors and very simple to implement.
- we can train a new NN to create a new embedding (more like probability) space ,using contractive loss methods such as Prototypical Learning.
### NOTE:
because i already spent much time on this home assignment (and i can continue doing so for longer), i skipped the specific implementation for the `lightweight_function()`.  

In [ ]:
class KNN_classifier: # this is a dummy implementation
    def __init__(self, k=3):
        self.k = k
    def fit(self, X, y):
        pass 
    def __call__(self, X):
        pass
    

## utility functions

In [7]:
def visualize(image, target, prediction):
    """visualize the image with the target and prediction boxes"""
    image = Image.fromarray(image.mul(255).permute(1, 2, 0).byte().numpy())
    draw = ImageDraw.Draw(image)
    for box in target["boxes"]:
        draw.rectangle(box.numpy(), outline="red")
    for box in prediction["boxes"]:
        draw.rectangle(box.numpy(), outline="green")
    image.show()

# Example
first we use my `CustomRCNN` class as the "of-the-shelf" `CORE_MODEL`.

the `raw_model.roi_heads.box_head.fc7` layer, is the final embedding layer for the proposed regions.

then we can envelope `CORE_MODEL` with our `BaseNN` class.

then we load our pickle `small_dataset` (which was previously generated using `data_utils.py` module)

In [8]:
raw_model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
CORE_MODEL = CustomRCNN(
    model=raw_model,
    embedding_layer_name="fc7",
    embedding_layer=raw_model.roi_heads.box_head.fc7,
)
base_model = BaseNN(CORE_MODEL)
base_dataset = BaseDataset.load_from_pkl()


/Users/ramshirazi/miniforge3/envs/abEnv0/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/ramshirazi/miniforge3/envs/abEnv0/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [16]:
classification_issue_dataset = identify_classification_issue(
    base_dataset, base_model, labels=[2, 4]
)
gt_label_name = label_map_dataset[int(classification_issue_dataset.targets[0]["labels"])]
pred_label_name = label_map_model[int(classification_issue_dataset.predictions[0]["labels"])]

visualize(
    classification_issue_dataset.images[0],
    classification_issue_dataset.targets[0],
    classification_issue_dataset.predictions[0],
)
print(f"Ground truth label: {gt_label_name}, Predicted label: {pred_label_name}")

Ground truth label: motorcycle, Predicted label: bicycle


In [17]:
gt_label_name = label_map_dataset[int(classification_issue_dataset.targets[1]["labels"])]
pred_label_name = label_map_model[int(classification_issue_dataset.predictions[1]["labels"])]

visualize(
    classification_issue_dataset.images[1],
    classification_issue_dataset.targets[1],
    classification_issue_dataset.predictions[1],
)
print(f"Ground truth label: {gt_label_name}, Predicted label: {pred_label_name}")

Ground truth label: bicycle, Predicted label: motorcycle


In [18]:
gt_label_name = label_map_dataset[int(classification_issue_dataset.targets[2]["labels"])]
pred_label_name = label_map_model[int(classification_issue_dataset.predictions[2]["labels"])]

visualize(
    classification_issue_dataset.images[2],
    classification_issue_dataset.targets[2],
    classification_issue_dataset.predictions[2],
)
print(f"Ground truth label: {gt_label_name}, Predicted label: {pred_label_name}")

Ground truth label: motorcycle, Predicted label: bicycle
